# SQL query from table names - Continued

In [15]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [4]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [9]:
#new
context_user = context.copy()
print(return_CCRMSQL("What is the average salary for employees?", context_user))

```sql
SELECT AVG(salary) AS average_salary
FROM employees;
```


In [10]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("What is the old employee?", old_context_user))

This is your SQL:
```sql
SELECT * 
FROM employees 
ORDER BY ID_usr ASC 
LIMIT 1;
```
This SQL query selects all columns from the employees table, orders the results by ID_usr in ascending order, and limits the output to the first row, which represents the oldest employee.


In [11]:
#new
print(return_CCRMSQL("What is the maxmum salary for employees?", context_user))

```sql
SELECT MAX(salary) AS max_salary
FROM employees;
```


In [13]:
#old
print(return_CCRMSQL("what is the champion of the month employees?", old_context_user))

This is your SQL:
```sql
SELECT name
FROM employees
ORDER BY ID_usr DESC
LIMIT 1;
```

This SQL query selects the name of the employee with the highest ID_usr, which could be considered the "champion of the month" based on the assumption that higher ID_usr values correspond to more recent entries.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [14]:
import openai


client = openai.OpenAI(api_key=OPENAI_API_KEY)

def ask_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

prompts = [
    "Generate an SQL query to get the highest salary from the employees table.",
    "Write an SQL query to count the number of employees in each department.",
    "Create an SQL query to retrieve the names of employees who joined after 2020."
]

results = {}
for i, prompt in enumerate(prompts, 1):
    print(f"Prompt {i}: {prompt}\n")
    response = ask_gpt(prompt)
    results[prompt] = response
    print(f"Response:\n{response}\n")
    print("-" * 50)

report = "### Report on GPT SQL Query Experiment\n\n"
for prompt, response in results.items():
    report += f"*Prompt:* {prompt}\n\n"
    report += f"*Response:*\nsql\n{response}\n\n\n"
    report += "-" * 50 + "\n"

with open("SQL_GPT_Report.txt", "w", encoding="utf-8") as file:
    file.write(report)

print("Report saved as 'SQL_GPT_Report.txt'.")

Prompt 1: Generate an SQL query to get the highest salary from the employees table.

Response:
Certainly! To retrieve the highest salary from an `employees` table, you can use the `MAX()` function in SQL. Here is the query:

```sql
SELECT MAX(salary) AS highest_salary
FROM employees;
```

This query will return the maximum value in the `salary` column, labeled as `highest_salary`, which represents the highest salary in the `employees` table.

--------------------------------------------------
Prompt 2: Write an SQL query to count the number of employees in each department.

Response:
To count the number of employees in each department, you can use the `GROUP BY` clause along with the `COUNT()` function in SQL. Here's a sample query that assumes you have an `employees` table with at least two columns: `department_id` and `employee_id`.

```sql
SELECT department_id, COUNT(employee_id) AS employee_count
FROM employees
GROUP BY department_id;
```

This query will return a list of departmen